# Day 2 - Lab 2: Documenting Key Decisions with ADRs (Solution)

**Objective:** Use an LLM as a research assistant to compare technical options and synthesize the findings into a formal, version-controlled Architectural Decision Record (ADR).

**Introduction:**
This solution notebook provides the complete prompts and explanations for the ADR generation lab. It demonstrates how to use an LLM for comparative research and then synthesize that research into a structured, formal document.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, recommended_models_table, prompt_enhancer

# Initialize separate LLM clients for different tasks using models from different providers.
# - Template generation: use a HuggingFace instruction-following model
# - Research/comparison: use an Anthropic model
# - Synthesis (final ADR): use another HuggingFace model to demonstrate multi-provider usage
template_client, template_model_name, template_api_provider = setup_llm_client(model_name="deepseek-ai/DeepSeek-V3.1")
research_client, research_model_name, research_api_provider = setup_llm_client(model_name="claude-opus-4-1-20250805")
synthesis_client, synthesis_model_name, synthesis_api_provider = setup_llm_client(model_name="gpt-5-2025-08-07")


/Users/agaleana/repos/AG-AISOFTDEV/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-21 21:13:25,422 ag_aisoftdev.utils INFO LLM Client configured provider=huggingface model=deepseek-ai/DeepSeek-V3.1 latency_ms=None artifacts_path=None
2025-09-21 21:13:25,704 ag_aisoftdev.utils INFO LLM Client configured provider=anthropic model=claude-opus-4-1-20250805 latency_ms=None artifacts_path=None
2025-09-21 21:13:25,863 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-5-2025-08-07 latency_ms=None artifacts_path=None


## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): The ADR Template

**Explanation:**
This prompt asks the LLM to generate a standard markdown template for an ADR. The key is to be specific about the sections required (`Title`, `Status`, `Context`, `Decision`, `Consequences`), which guides the LLM to produce a well-structured and reusable template.

In [3]:
adr_template_prompt = """You are a principal engineer who champions clear documentation. Generate a concise, reusable markdown template for an Architectural Decision Record (ADR).

The template must include the following sections:
- # Title: [A short, descriptive title for the decision]
- **Status:** [Proposed | Accepted | Deprecated | Superseded]
- ## Context
  - [Describe the problem, the driving forces, and the constraints.]
- ## Decision
  - [State the chosen solution clearly and concisely.]
- ## Consequences
  - [List the positive outcomes, negative trade-offs, and any future work required.]"""

print("--- Generating ADR Template ---")
enhanced_adr_template_prompt = prompt_enhancer(adr_template_prompt)
adr_template_content = get_completion(enhanced_adr_template_prompt, template_client, template_model_name, template_api_provider)
print(adr_template_content)

if adr_template_content:
    save_artifact(adr_template_content, "templates/adr_template.md", overwrite=True)


2025-09-21 21:14:34,860 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Generating ADR Template ---
```markdown
# Title: [A short, descriptive title for the decision]

**Status:** [Proposed | Accepted | Deprecated | Superseded]

## Context
- [Describe the problem, the driving forces, and the constraints.]

## Decision
- [State the chosen solution clearly and concisely.]

## Consequences
- [List the positive outcomes, negative trade-offs, and any future work required.]
```


### Challenge 2 (Intermediate): AI-Assisted Research

**Explanation:**
This prompt leverages the LLM's vast training data to perform a comparative analysis. By instructing it to be an "unbiased research assistant" and asking for "pros and cons for each," we guide the model to provide a balanced view rather than a simple recommendation. This produces a more valuable and objective input for our own decision-making process.

In [4]:
db_research_prompt = """You are an unbiased research assistant. Your task is to provide a balanced technical comparison for a software development team.

For the use case of a new hire onboarding tool that needs a semantic search feature, compare and contrast the following two approaches:

1.  **Approach 1:** Using PostgreSQL with the `pgvector` extension.
2.  **Approach 2:** Using a specialized, dedicated vector database (e.g., ChromaDB, FAISS, Weaviate).

Please provide a summary of the pros and cons for each approach. Consider factors like operational complexity, cost, query flexibility, and scalability for a small-to-medium sized enterprise application.
"""

print("--- Researching Database Options ---")
enhanced_db_research_prompt = prompt_enhancer(db_research_prompt)
db_research_output = get_completion(enhanced_db_research_prompt, research_client, research_model_name, research_api_provider)
print(db_research_output)


2025-09-21 21:14:44,772 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Researching Database Options ---
### Comparative Analysis

| Factor | PostgreSQL + pgvector – Pros | PostgreSQL + pgvector – Cons | Dedicated Vector DB – Pros | Dedicated Vector DB – Cons |
|--------|-----------------------------|-----------------------------|---------------------------|----------------------------|
| **Operational Complexity** | • Single database to manage, backup, and monitor<br>• Existing PostgreSQL expertise applies<br>• Unified security model and access controls<br>• Simple disaster recovery with standard PostgreSQL tools | • Requires PostgreSQL tuning knowledge for vector workloads<br>• Index rebuilding can block operations<br>• Limited vector-specific monitoring tools | • Purpose-built admin interfaces for vector operations<br>• Automated index optimization<br>• Built-in vector-specific monitoring dashboards | • Additional system to operate and secure<br>• Separate backup/recovery procedures needed<br>• New technology stack for team to learn<br>• Complex dat

### Challenge 3 (Advanced): Synthesizing the ADR

**Explanation:**
This prompt demonstrates a powerful synthesis task. We provide the LLM with two key inputs: unstructured information (the research) and a desired structure (the template). The agent's job is to merge them, creating a polished, formal document. This is a repeatable pattern for turning raw analysis into professional documentation. By assigning the persona of a Staff Engineer, we guide the LLM to adopt a formal and authoritative tone suitable for an official project artifact.

In [5]:
adr_template = load_artifact("templates/adr_template.md")

synthesis_prompt = f"""You are a Staff Engineer responsible for documenting key architectural decisions.

Your task is to populate the provided ADR template to formally document the decision to **use PostgreSQL with the pgvector extension** for our project.

Use the research provided below to fill in the 'Context' and 'Consequences' sections of the template. Be thorough and objective, summarizing the key points from the research.

--- ADR TEMPLATE ---
{adr_template}
--- END TEMPLATE ---

--- RESEARCH CONTEXT ---
{db_research_output}
--- END CONTEXT ---

The final ADR should be complete and ready for review.
"""

print("--- Synthesizing Final ADR ---")
if adr_template and 'db_research_output' in locals() and db_research_output:
    enhanced_synthesis_prompt = prompt_enhancer(synthesis_prompt)
    final_adr = get_completion(enhanced_synthesis_prompt, synthesis_client, synthesis_model_name, synthesis_api_provider)
    print(final_adr)
    save_artifact(final_adr, "adr_001_database_choice.md")
else:
    print("Skipping ADR synthesis because template or research is missing.")


2025-09-21 21:15:22,062 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Synthesizing Final ADR ---
```markdown
# Title: Use PostgreSQL with pgvector for Vector Search and Storage

**Status:** Accepted

## Context
- We need vector similarity search tightly integrated with relational data, and must choose between PostgreSQL with pgvector and a dedicated vector database.
- Operational simplicity and cost control are priorities; we prefer to avoid new licenses, vendors, and parallel operational stacks while leveraging existing PostgreSQL expertise and security.
- Target workload is small-to-medium scale (approximately 1–10 million embeddings) with acceptable latency under ~300 ms; the ability to perform native SQL joins alongside vector queries is important.
- Research indicates PostgreSQL + pgvector provides single-stack simplicity, unified security, and native SQL joins with vectors, delivering adequate performance at the targeted scale after tuning.
- Risks include limited horizontal scaling, fewer vector search algorithms, heavier compute demands under

## Lab Conclusion

Well done! You have used an LLM to automate a complex but critical part of the architectural process. You leveraged its vast knowledge base for research and then used it again for synthesis, turning raw analysis into a formal, structured document. This `adr_001_database_choice.md` file now serves as a permanent, valuable record for anyone who works on this project in the future.

> **Key Takeaway:** The pattern of **Research -> Synthesize -> Format** is a powerful workflow. You can use an LLM to gather unstructured information and then use it again to pour that information into a structured template, creating high-quality, consistent documentation with minimal effort.